## 1 Realizar deploy da base dados em anexo em um bucket/GCP via colab

In [37]:
# imports
import os
import io
import pandas as pd
import mysql.connector
from mysql.connector import Error
from google.cloud import storage
from pyspark.sql import DataFrame, SparkSession
from unidecode import unidecode

from sqlalchemy import Column, Integer, Float, String, Boolean, DateTime, Text
from sqlalchemy.orm import declarative_base


In [2]:
def clean_column_names(df):

    if isinstance(df, DataFrame):
        for col in df.columns:
            df = df.withColumnRenamed(col, unidecode(col.lower().replace(' ', '_')))
    else:
        for col in df.columns:
            df.rename(columns={col:unidecode(col.lower().replace(' ', '_'))}, inplace=True)

    return df

In [3]:
# credencias GCP
credentials_path = "/home/jovyan/code/.devcontainer/soulcode-iam-key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path


In [4]:
# constantes
BUCKET_NAME =  'soulcode-analytics-data-bootcamp'
NAME_FILE = '../dataset/balanco_petro.xlsx'
DEST_BLOB_NAME = 'raw_data/balanco_petro.csv'
# DEST_BLOB_NAME_PROCESSED = 'clean_data/lista-de-dados-processed.csv'

In [ ]:
# Conectar-se ao GCP e fazer upload de arquivo

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

df = pd.read_excel(NAME_FILE)
df.to_csv("/tmp/converted.csv", index=False)
upload_blob(BUCKET_NAME, "/tmp/converted.csv" , DEST_BLOB_NAME)

## Consumir a base armazenada no bucket via colab e Realizar ETL ou ELT com pandas e/ou pyspark

In [ ]:
def download_blob_into_memory(bucket_name, blob_name):
    """Downloads a blob into memory."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(blob_name)
    contents = blob.download_as_bytes()

    print(
        "Downloaded storage object {} from bucket {} as the following bytes object: {}.".format(
            blob_name, bucket_name, contents.decode("utf-8")
        )
    )
    return contents

data = download_blob_into_memory(BUCKET_NAME, DEST_BLOB_NAME)

df = pd.read_csv(io.BytesIO(data))




In [7]:
df.columns

Index(['XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data',
       'Balanço Patrimonial - PETROBRAS', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52',
       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnam

In [6]:
df.head()

,"XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data",Balanço Patrimonial - PETROBRAS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,31/03/2009,30/06/2009,30/09/2009,31/12/2009,31/03/2010,30/06/2010,30/09/2010,31/12/2010,31/03/2011,30/06/2011,...,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024
1,Ativo Total,305265344.512,333789528.064,345607274.496,365998080,382029856.768,507697037.312,519970029.568,544945274.88,554583457.792,...,972950994.944,998662012.928,1004753977.344,947574013.952,976708960.256,978576998.4,990459002.88,1025495990.272,1050887979.008,1067292033.024
2,Ativo Circulante,57621536.768,75719057.408,76674015.232,74459103.232,71980236.8,111415033.856,106685161.472,120036376.576,119493476.352,...,168247001.088,199511998.464,224784990.208,159327010.816,163051995.136,157193994.24,136816001.024,147311001.6,157079011.328,165964005.376
3,Caixa e Equivalentes de Caixa,10072162.304,30088286.208,28795713.536,26951325.696,24209866.752,47291932.672,30323259.392,43344818.176,34672615.424,...,58410000.384,81601003.52,85310996.48,23650000.896,41722998.784,52276998.144,49882001.408,60642000.896,61612998.656,57689001.984
4,Aplicações Financeiras,0,0,0,0,0,0,26017296.384,20015771.648,24969263.104,...,3630000.128,5967000.064,14956999.68,13038000.128,14469999.616,14629000.192,11102999.552,6504999.936,13649999.872,24071999.488


In [7]:
df = df.transpose()
# df.to_csv("/tmp/transpose.csv", index=False)
# upload_blob(BUCKET_NAME, "/tmp/transpose.csv" , 'clean_data/transpose.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
"XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data",31/03/2009,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
Balanço Patrimonial - PETROBRAS,30/06/2009,305265344.512,57621536.768,10072162.304,0,14555268.096,19674548.224,0,0,0,...,149847965.696,78966685.696,514856.992,9920,58865377.28,10918406.144,572716.032,0,0,0
Unnamed: 2,30/09/2009,333789528.064,75719057.408,30088286.208,0,13643311.104,20635084.8,0,0,0,...,155391688.704,78966685.696,514856.992,9718,58866671.616,16466363.392,567398.976,0,0,0
Unnamed: 3,31/12/2009,345607274.496,76674015.232,28795713.536,0,13984270.336,21424652.288,0,0,0,...,159464587.264,78966685.696,514856.992,350,79521013.76,0,461687.008,0,0,0
Unnamed: 4,31/03/2010,365998080,74459103.232,26951325.696,0,16200354.816,20030609.408,0,0,0,...,170299080.704,78966685.696,514856.992,0,84879695.872,5971459.072,-33617,0,0,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data to Unnamed: 60
Data columns (total 56 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       61 non-null     object
 1   1       61 non-null     object
 2   2       61 non-null     object
 3   3       61 non-null     object
 4   4       61 non-null     object
 5   5       61 non-null     object
 6   6       61 non-null     object
 7   7       61 non-null     object
 8   8       61 non-null     object
 9   9       61 non-null     object
 10  10      61 non-null     object
 11  11      61 non-null     object
 12  12      61 non-null     object
 13  13      61 non-null     object
 14  14      61 non-null     object
 15  15      61 non-null     object
 16  16      61 non-null     object
 17  17      61 non-null     object
 18  18      61 non-null     object
 19  19      61 non-null     object
 20  20      61 non-null     object
 21  21      61 no

In [9]:
df.columns

RangeIndex(start=0, stop=56, step=1)

In [10]:
index = [ x for x in range(len(df))]
len(index)

61

In [11]:
df.index = index
df

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,31/03/2009,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2009,305265344.512,57621536.768,10072162.304,0,14555268.096,19674548.224,0,0,0,...,149847965.696,78966685.696,514856.992,9920,58865377.28,10918406.144,572716.032,0,0,0
2,30/09/2009,333789528.064,75719057.408,30088286.208,0,13643311.104,20635084.8,0,0,0,...,155391688.704,78966685.696,514856.992,9718,58866671.616,16466363.392,567398.976,0,0,0
3,31/12/2009,345607274.496,76674015.232,28795713.536,0,13984270.336,21424652.288,0,0,0,...,159464587.264,78966685.696,514856.992,350,79521013.76,0,461687.008,0,0,0
4,31/03/2010,365998080,74459103.232,26951325.696,0,16200354.816,20030609.408,0,0,0,...,170299080.704,78966685.696,514856.992,0,84879695.872,5971459.072,-33617,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,31/03/2023,978576998.4,157193994.24,52276998.144,14629000.192,23497000.96,40483000.32,0,6102000.128,0,...,402083980.288,205431996.416,3102000.128,0,166717997.056,0,0,0,26831998.976,0
57,30/06/2023,990459002.88,136816001.024,49882001.408,11102999.552,21041000.448,39240998.912,0,6352999.936,0,...,370682983.552,205431996.416,3102000.128,0,135016988.672,0,0,0,27132000.256,0
58,30/09/2023,1025495990.272,147311001.6,60642000.896,6504999.936,25501999.104,39509999.616,0,5965000.192,0,...,386006988.544,205431996.416,2131000.064,0,146422005.76,0,0,0,32021999.616,0
59,31/12/2023,1050887979.008,157079011.328,61612998.656,13649999.872,29702000.64,37184000,0,5703000.064,0,...,380441005.824,205431996.416,-538000,0,159171002.368,0,0,0,16376000.512,0


In [12]:
columns = df.iloc[0].to_list()
df.columns = columns
df.rename(columns={df.columns[0]: "Data"}, inplace=True)
df

,Data,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
0,31/03/2009,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2009,305265344.512,57621536.768,10072162.304,0,14555268.096,19674548.224,0,0,0,...,149847965.696,78966685.696,514856.992,9920,58865377.28,10918406.144,572716.032,0,0,0
2,30/09/2009,333789528.064,75719057.408,30088286.208,0,13643311.104,20635084.8,0,0,0,...,155391688.704,78966685.696,514856.992,9718,58866671.616,16466363.392,567398.976,0,0,0
3,31/12/2009,345607274.496,76674015.232,28795713.536,0,13984270.336,21424652.288,0,0,0,...,159464587.264,78966685.696,514856.992,350,79521013.76,0,461687.008,0,0,0
4,31/03/2010,365998080,74459103.232,26951325.696,0,16200354.816,20030609.408,0,0,0,...,170299080.704,78966685.696,514856.992,0,84879695.872,5971459.072,-33617,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,31/03/2023,978576998.4,157193994.24,52276998.144,14629000.192,23497000.96,40483000.32,0,6102000.128,0,...,402083980.288,205431996.416,3102000.128,0,166717997.056,0,0,0,26831998.976,0
57,30/06/2023,990459002.88,136816001.024,49882001.408,11102999.552,21041000.448,39240998.912,0,6352999.936,0,...,370682983.552,205431996.416,3102000.128,0,135016988.672,0,0,0,27132000.256,0
58,30/09/2023,1025495990.272,147311001.6,60642000.896,6504999.936,25501999.104,39509999.616,0,5965000.192,0,...,386006988.544,205431996.416,2131000.064,0,146422005.76,0,0,0,32021999.616,0
59,31/12/2023,1050887979.008,157079011.328,61612998.656,13649999.872,29702000.64,37184000,0,5703000.064,0,...,380441005.824,205431996.416,-538000,0,159171002.368,0,0,0,16376000.512,0


In [13]:
df = df.drop(0, axis=0)


In [14]:
df.head()

,Data,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2009,305265344.512,57621536.768,10072162.304,0,14555268.096,19674548.224,0,0,0,...,149847965.696,78966685.696,514856.992,9920,58865377.28,10918406.144,572716.032,0,0,0
2,30/09/2009,333789528.064,75719057.408,30088286.208,0,13643311.104,20635084.8,0,0,0,...,155391688.704,78966685.696,514856.992,9718,58866671.616,16466363.392,567398.976,0,0,0
3,31/12/2009,345607274.496,76674015.232,28795713.536,0,13984270.336,21424652.288,0,0,0,...,159464587.264,78966685.696,514856.992,350,79521013.76,0,461687.008,0,0,0
4,31/03/2010,365998080,74459103.232,26951325.696,0,16200354.816,20030609.408,0,0,0,...,170299080.704,78966685.696,514856.992,0,84879695.872,5971459.072,-33617,0,0,0
5,30/06/2010,382029856.768,71980236.8,24209866.752,0,15961581.568,19680464.896,0,0,0,...,176974135.296,85108547.584,0,0,79252692.992,12494328.832,118554,0,0,0


In [15]:
df.tail()

,Data,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
56,31/03/2023,978576998.4,157193994.24,52276998.144,14629000.192,23497000.96,40483000.32,0,6102000.128,0,...,402083980.288,205431996.416,3102000.128,0,166717997.056,0,0,0,26831998.976,0
57,30/06/2023,990459002.88,136816001.024,49882001.408,11102999.552,21041000.448,39240998.912,0,6352999.936,0,...,370682983.552,205431996.416,3102000.128,0,135016988.672,0,0,0,27132000.256,0
58,30/09/2023,1025495990.272,147311001.6,60642000.896,6504999.936,25501999.104,39509999.616,0,5965000.192,0,...,386006988.544,205431996.416,2131000.064,0,146422005.76,0,0,0,32021999.616,0
59,31/12/2023,1050887979.008,157079011.328,61612998.656,13649999.872,29702000.64,37184000,0,5703000.064,0,...,380441005.824,205431996.416,-538000,0,159171002.368,0,0,0,16376000.512,0
60,31/03/2024,1067292033.024,165964005.376,57689001.984,24071999.488,25184000,40848998.4,0,7701000.192,0,...,407635978.24,205431996.416,-1684999.936,0,182872997.888,0,0,0,21016000.512,0


In [16]:
df.shape


(60, 56)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 1 to 60
Data columns (total 56 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   Data                                                          60 non-null     object
 1   Ativo Total                                                   60 non-null     object
 2   Ativo Circulante                                              60 non-null     object
 3   Caixa e Equivalentes de Caixa                                 60 non-null     object
 4   Aplicações Financeiras                                        60 non-null     object
 5   Contas a Receber                                              60 non-null     object
 6   Estoques                                                      60 non-null     object
 7   Ativos Biológicos                                             60 non-null     object
 8

In [18]:
# verificando colunas com valores nulos
df.isnull().sum()

Data                                                            0
Ativo Total                                                     0
Ativo Circulante                                                0
Caixa e Equivalentes de Caixa                                   0
Aplicações Financeiras                                          0
Contas a Receber                                                0
Estoques                                                        0
Ativos Biológicos                                               0
Tributos a Recuperar                                            0
Despesas Antecipadas                                            0
Outros Ativos Circulantes                                       0
Ativo Realizável a Longo Prazo                                  0
Aplicações Financeiras Avaliadas a Valor Justo                  0
Aplicações Financeiras Avaliadas ao Custo Amortizado            0
Contas a Receber                                                0
Estoques  

In [19]:
# verificar valores zerados
df.eq(0).sum()


Data                                                            0
Ativo Total                                                     0
Ativo Circulante                                                0
Caixa e Equivalentes de Caixa                                   0
Aplicações Financeiras                                          0
Contas a Receber                                                0
Estoques                                                        0
Ativos Biológicos                                               0
Tributos a Recuperar                                            0
Despesas Antecipadas                                            0
Outros Ativos Circulantes                                       0
Ativo Realizável a Longo Prazo                                  0
Aplicações Financeiras Avaliadas a Valor Justo                  0
Aplicações Financeiras Avaliadas ao Custo Amortizado            0
Contas a Receber                                                0
Estoques  

In [22]:
# eliminar colunas que todos os valores estão zerado
cols_to_drop = df.eq(0).all(axis=0)
cols_to_drop

Data                                                            False
Ativo Total                                                     False
Ativo Circulante                                                False
Caixa e Equivalentes de Caixa                                   False
Aplicações Financeiras                                          False
Contas a Receber                                                False
Estoques                                                        False
Ativos Biológicos                                               False
Tributos a Recuperar                                            False
Despesas Antecipadas                                            False
Outros Ativos Circulantes                                       False
Ativo Realizável a Longo Prazo                                  False
Aplicações Financeiras Avaliadas a Valor Justo                  False
Aplicações Financeiras Avaliadas ao Custo Amortizado            False
Contas a Receber    

In [23]:
df.columns

Index(['Data', 'Ativo Total', 'Ativo Circulante',
       'Caixa e Equivalentes de Caixa', 'Aplicações Financeiras',
       'Contas a Receber', 'Estoques', 'Ativos Biológicos',
       'Tributos a Recuperar', 'Despesas Antecipadas',
       'Outros Ativos Circulantes', 'Ativo Realizável a Longo Prazo',
       'Aplicações Financeiras Avaliadas a Valor Justo',
       'Aplicações Financeiras Avaliadas ao Custo Amortizado',
       'Contas a Receber', 'Estoques', 'Ativos Biológicos',
       'Tributos Diferidos', 'Despesas Antecipadas',
       'Créditos com Partes Relacionadas', 'Outros Ativos Não Circulantes',
       'Investimentos', 'Imobilizado', 'Intangível', 'Diferido',
       'Passivo Total', 'Passivo Circulante',
       'Obrigações Sociais e Trabalhistas', 'Fornecedores',
       'Obrigações Fiscais', 'Empréstimos e Financiamentos',
       'Passivos com Partes Relacionadas', 'Dividendos e JCP a Pagar',
       'Outros', 'Provisões',
       'Passivos sobre Ativos Não-Correntes a Venda e Des

In [24]:
df.drop(df.columns[cols_to_drop], axis=1, inplace=True)


In [25]:
df

,Data,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2009,305265344.512,57621536.768,10072162.304,0,14555268.096,19674548.224,0,0,0,...,149847965.696,78966685.696,514856.992,9920,58865377.28,10918406.144,572716.032,0,0,0
2,30/09/2009,333789528.064,75719057.408,30088286.208,0,13643311.104,20635084.8,0,0,0,...,155391688.704,78966685.696,514856.992,9718,58866671.616,16466363.392,567398.976,0,0,0
3,31/12/2009,345607274.496,76674015.232,28795713.536,0,13984270.336,21424652.288,0,0,0,...,159464587.264,78966685.696,514856.992,350,79521013.76,0,461687.008,0,0,0
4,31/03/2010,365998080,74459103.232,26951325.696,0,16200354.816,20030609.408,0,0,0,...,170299080.704,78966685.696,514856.992,0,84879695.872,5971459.072,-33617,0,0,0
5,30/06/2010,382029856.768,71980236.8,24209866.752,0,15961581.568,19680464.896,0,0,0,...,176974135.296,85108547.584,0,0,79252692.992,12494328.832,118554,0,0,0
6,30/09/2010,507697037.312,111415033.856,47291932.672,0,18407325.696,21359228.928,0,0,0,...,297360982.016,200160870.4,164696,0,77830742.016,19251855.36,-47178,0,0,0
7,31/12/2010,519970029.568,106685161.472,30323259.392,26017296.384,17333975.04,19815675.904,0,8934797.312,1006419.008,...,306765188.864,205357105.152,-6257,0,101323726.848,0,287084,-196479.008,0,0
8,31/03/2011,544945274.88,120036376.576,43344818.176,20015771.648,17777672.192,23868891.136,0,10178227.2,1282752,...,314781563.904,205357105.152,1847,0,101323726.848,8376065.024,177879.008,-455044.992,0,0
9,30/06/2011,554583457.792,119493476.352,34672615.424,24969263.104,18762930.176,25751416.832,0,11322895.36,1218744.064,...,322711576.064,205379731.456,48674,0,101301100.544,16710102.016,144754,-872763.008,0,0
10,30/09/2011,582124896.256,120493400.064,33659195.392,21410791.424,20466372.608,27397234.688,0,12568754.176,1449747.968,...,328404782.08,205379731.456,101923,0,101301100.544,20445116.416,250479.008,926436.992,0,0


## Inserir os dados do DataFrame via colab no banco de dados MySQL.

In [26]:
HOST = '34.16.33.165'
DB = 'petrobras'
USER = 'soulcode'
PASSWORD = 'Soulcode@!12pass'

In [86]:
def connect_mysql(servidor, basededados, usuario, senha):
    try:
        conn = mysql.connector.connect(
            host=servidor,
            database=basededados,
            user=usuario,
            password=senha
        )
        if conn.is_connected():
            print("Conexão bem-sucedida ao banco de dados MySQL")
            return conn
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None
        
# Função para criar o banco de dados
def create_db(conexao, basededados):
    try:
        cursor = conexao.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {basededados}")
        conexao.commit()
        print(f"Banco de dados {basededados} criado ou já existente.")
    except Error as e:
        print(f"Erro ao criar banco de dados: {e}")



def create_table(conexao,sql):
    try:
        cursor = conexao.cursor()
        cursor.execute(sql)
        conexao.commit()
        print("Tabela criada ou já existente.")
    except Error as e:
        print(f"Erro ao criar tabela: {e}")
        
# Função para inserir dados do DataFrame na tabela
def insert_data_df(df, conexao):
    cursor = conexao.cursor()
    for _, row in df.iterrows():
        values = ', '.join(f"'{str(val)}'" for val in row)
        insert_query = f"INSERT INTO petro_data ({', '.join(df.columns)}) VALUES ({values})"
        try:
            cursor.execute(insert_query)
        except Error as e:
            print(f"Erro ao inserir dados: {e}")
    conexao.commit()
    print("Dados inseridos com sucesso.")

# Mapeamento de tipos do Pandas para MySQL
def mapear_tipo_para_mysql(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_string_dtype(dtype):
        return 'VARCHAR(255)'  # Definir tamanho adequado
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'  # Padrão para outros tipos

# Gerar declaração de criação de tabela para MySQL
def gerar_sql_criacao_tabela(df, nome_tabela):
    colunas_sql = []
    for coluna in df.columns:
        tipo_mysql = mapear_tipo_para_mysql(df[coluna].dtype)
        colunas_sql.append(f"`{coluna}` {tipo_mysql}")

    colunas_sql_str = ",\n  ".join(colunas_sql)
    sql_criacao_tabela = f"CREATE TABLE IF NOT EXISTS `{nome_tabela}` (\n  {colunas_sql_str}\n);"
    return sql_criacao_tabela

# Exemplo de uso
nome_tabela = 'petro_data'
sql = gerar_sql_criacao_tabela(df, nome_tabela)
print(sql)

CREATE TABLE IF NOT EXISTS `petro_data` (
  `data` VARCHAR(255),
  `ativo_total` VARCHAR(255),
  `ativo_circulante` VARCHAR(255),
  `caixa_e_equivalentes_de_caixa` VARCHAR(255),
  `aplicacoes_financeiras` VARCHAR(255),
  `tributos_a_recuperar` VARCHAR(255),
  `outros_ativos_circulantes` VARCHAR(255),
  `ativo_realizavel_a_longo_prazo` VARCHAR(255),
  `aplicacoes_financeiras_avaliadas_a_valor_justo` VARCHAR(255),
  `aplicacoes_financeiras_avaliadas_ao_custo_amortizado` VARCHAR(255),
  `creditos_com_partes_relacionadas` VARCHAR(255),
  `outros_ativos_nao_circulantes` VARCHAR(255),
  `investimentos` VARCHAR(255),
  `imobilizado` VARCHAR(255),
  `intangivel` VARCHAR(255),
  `diferido` VARCHAR(255),
  `passivo_total` VARCHAR(255),
  `passivo_circulante` VARCHAR(255),
  `obrigacoes_sociais_e_trabalhistas` VARCHAR(255),
  `fornecedores` VARCHAR(255),
  `obrigacoes_fiscais` VARCHAR(255),
  `dividendos_e_jcp_a_pagar` VARCHAR(255),
  `passivo_nao_circulante` VARCHAR(255),
  `lucros_e_receitas_a_

In [77]:
conexion = connect_mysql(HOST,DB,USER,PASSWORD)
conexion

Conexão bem-sucedida ao banco de dados MySQL


In [54]:
create_db(conexion,DB)

Banco de dados petrobras criado ou já existente.


In [ ]:
df = clean_column_names(df)
cols = df.columns
duplicated_cols = cols[cols.duplicated()]
df.drop(df[duplicated_cols], axis=1, inplace=True)
df



In [80]:
create_table(conexion,sql)

Tabela criada ou já existente.


In [85]:
insert_data_df(df, conexion)

Erro ao inserir dados: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '/prejuizos_acumulados, ajustes_de_avaliacao_patrimonial, ajustes_acumulados_de_c' at line 1
Erro ao inserir dados: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '/prejuizos_acumulados, ajustes_de_avaliacao_patrimonial, ajustes_acumulados_de_c' at line 1
Erro ao inserir dados: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '/prejuizos_acumulados, ajustes_de_avaliacao_patrimonial, ajustes_acumulados_de_c' at line 1
Erro ao inserir dados: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '/prejuizos_acumulados, ajustes_de_av

In [50]:
#TODO2: utilizar sqlalchemy(Encontrei problemas ao gerar conexao com a mmysql GCP -> aprender

Base = declarative_base()

def mapear_tipo_para_sqlalchemy(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return Integer
    elif pd.api.types.is_float_dtype(dtype):
        return Float
    elif pd.api.types.is_bool_dtype(dtype):
        return Boolean
    elif pd.api.types.is_string_dtype(dtype):
        return String(255)  # Definir tamanho adequado
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return DateTime
    else:
        return Text 



def gerar_classe_sqlalchemy(df, nome_classe):
    # Definir dinamicamente uma nova classe para o SQLAlchemy
    attrs = {
        '__tablename__': nome_classe.lower(),
        'id': Column(Integer, primary_key=True, autoincrement=True)
    }

    # Para cada coluna no DataFrame, mapear o tipo de dados
    for col in df.columns:
        dtype = df[col].dtype
        tipo_sqlalchemy = mapear_tipo_para_sqlalchemy(dtype)
        print(f"Coluna: {col}, Tipo SQLAlchemy: {tipo_sqlalchemy}")
        
        # Adicionar dinamicamente a coluna à classe
        attrs[col] = Column(tipo_sqlalchemy)pymysql

    # Criar uma classe SQLAlchemy dinamicamente
    return type(nome_classe, (Base,), attrs)





/tmp/ipykernel_126/3606643353.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Referencias

- https://cloud.google.com/python/docs/setup (environment)
- 